In [1]:
!pip install imageio==2.27.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.0 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install tensorflow_addons==0.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.9 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install tensorflow_datasets==4.8.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.6/120.6 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 KB 5.4 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=c4e1d21cc010a3b848e9559f811a5b022dfbba880e0048dd7065fcbe7e23f7cd
  Stored in directory: /root/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install mat73==0.60

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [7]:
import glob
import os
import pathlib
import time
import zipfile
import imageio
import scipy.io as sio
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
from PIL import Image
import requests
import scipy.ndimage as ndimage
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, Dense, Dropout, GaussianNoise
from tensorflow.keras.layers import Flatten, Input, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from scipy.io import loadmat
import mat73
import tqdm

%matplotlib inline

In [27]:
matfiles = []
tensors = []
for file in glob.glob('data/*.mat'):
    mtfile = mat73.loadmat(file)
    mat = {}
    for key, value in mtfile['cjdata'].items():
        mat[key] = value
        matfiles.append(mat)
matfiles_df = pd.DataFrame(matfiles)
training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(matfiles_df['image'].to_list(), tf.float32),
            tf.cast(matfiles_df['label'].to_list(), tf.float32),
            tf.cast(matfiles_df['tumorBorder'].to_list(), tf.float32),
            tf.cast(matfiles_df['tumorMask'].to_list(), tf.float32),
        )
    )
)


<TensorSliceDataset element_spec=(TensorSpec(shape=(512, 512), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(62,), dtype=tf.float32, name=None), TensorSpec(shape=(512, 512), dtype=tf.float32, name=None))>
(<tf.Tensor: shape=(512, 512), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>, <tf.Tensor: shape=(62,), dtype=float32, numpy=
array([219.17938, 205.12943, 229.71684, 210.7494 , 233.93184, 219.17938,
       240.95682, 231.82434, 247.2793 , 236.03934, 254.30428, 245.8743 ,
       254.30428, 248.6843 , 252.89928, 255.00677, 248.6843 , 262.73425,
       241.65932, 264.13925, 240.95682, 267.65173, 236.74184, 269.75925,
       227.60936, 273.27173, 223.39436, 276

In [0]:
fig, axes= plt.subplots(6,4,figsize=(18,30))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(matfiles[i]['image'])
    ax.set_title(matfiles[i]['PID'])
    ax.axis('off')

In [0]:
print("Sum of NA values:", sum([matfiles[c]=='NaN' for c in range(len(matfiles))]))
unique, counts = np.unique([matfiles[c]['label'] for c in range(len(matfiles))], return_counts=True)
print("Count of unique label values:", unique,counts)
class_count_0, class_count_1, class_count_2 = matfiles[c].value_counts()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a7cd613f-660f-4c4e-8aa3-b044f6915c0f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>